In [3]:
from os import listdir
from os.path import join, isfile

In [4]:
mypath = './datasets/catsvsdogs/images/'

file_name = [f for f in listdir(mypath) if isfile(join(mypath,f))]
print(len(file_name))

3002


In [5]:
import os
import sys
import cv2
import numpy as np
import shutil

In [7]:
dir_train = './datasets/catsvsdogs/train/'
dir_test = './datasets/catsvsdogs/test/'

def make_dir(directory):
    if os.path.exists(directory):
        shutil.rmtree(directory)
    os.makedirs(directory)
make_dir(dir_test)
make_dir(dir_train)

dog_count = 0
cat_count = 0
X_train = []
y_train = []
X_test = []
y_test = []

train_size = 2000
test_size = 500

for i, file in enumerate(file_name):
    if file[0] == 'd':
        dog_count += 1
        image = cv2.imread(mypath + file)
        image = cv2.resize(image,(150,150),interpolation = cv2.INTER_AREA)
        if dog_count + cat_count <= train_size:
            cv2.imwrite(dir_train + 'dog' + str(dog_count) + '.jpg',image)
            X_train.append(image)
            y_train.append(1)
        else:
            if dog_count + cat_count <= train_size + test_size:
                cv2.imwrite(dir_test + 'dog' + str(dog_count) + '.jpg', image)
                X_test.append(image)
                y_test.append(1)
                
    if file[0] == 'c':
        cat_count += 1
        image = cv2.imread(mypath + file)
        image = cv2.resize(image,(150,150),interpolation = cv2.INTER_AREA)
        if dog_count + cat_count <= train_size:
            cv2.imwrite(dir_train + 'cat' + str(cat_count) + '.jpg',image)
            X_train.append(image)
            y_train.append(0)
        else:
            if dog_count + cat_count <= train_size + test_size:
                cv2.imwrite(dir_test + 'cat' + str(cat_count) + '.jpg', image)
                X_test.append(image)
                y_test.append(0)

In [10]:
cv2.imshow('Ori',X_train[2])
cv2.waitKey(0)
cv2.destroyAllWindows()

In [11]:
len(X_train)

2000

In [12]:
X_train = np.array(X_train)

In [14]:
X_test = np.array(X_test)
y_train = np.array(y_train)
y_test = np.array(y_test)

In [27]:
X_test = X_test.astype('float32')
X_train = X_train.astype('float32')
X_test /= 255
X_train /= 255 

In [29]:
X_train.sha

(2000, 150, 150, 3)

In [17]:
import keras

In [21]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, MaxPooling2D, Flatten, Conv2D

In [22]:
from keras.optimizers import SGD

In [50]:
model = Sequential()

model.add(Conv2D(16,(3,3),activation='relu',input_shape = (X_test[1].shape)))
model.add(Conv2D(32,(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(64,(3,3),activation='relu',input_shape = (X_test[1])))
model.add(MaxPooling2D((2,2)))
model.add(Conv2D(64,(3,3),activation='relu',input_shape = (X_test[1])))
model.add(MaxPooling2D((2,2)))


model.add(Dropout(0.25))
model.add(Flatten())

model.add(Dense(128,activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(1,activation='sigmoid'))
          
model.compile(loss='binary_crossentropy', metrics='accuracy')

In [46]:
print(model.summary())

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_27 (Conv2D)           (None, 148, 148, 16)      448       
_________________________________________________________________
conv2d_28 (Conv2D)           (None, 146, 146, 32)      4640      
_________________________________________________________________
max_pooling2d_15 (MaxPooling (None, 73, 73, 32)        0         
_________________________________________________________________
conv2d_29 (Conv2D)           (None, 71, 71, 64)        18496     
_________________________________________________________________
max_pooling2d_16 (MaxPooling (None, 35, 35, 64)        0         
_________________________________________________________________
conv2d_30 (Conv2D)           (None, 33, 33, 64)        36928     
_________________________________________________________________
max_pooling2d_17 (MaxPooling (None, 16, 16, 64)      

In [51]:
model.fit(X_train,y_train,batch_size=16,epochs=5,validation_data=(X_test,y_test))

Epoch 1/5
125/125 [==============================] - 33s 267ms/step - loss: 0.7073 - accuracy: 0.5385 - val_loss: 0.6566 - val_accuracy: 0.5980
Epoch 2/5
125/125 [==============================] - 33s 260ms/step - loss: 0.6696 - accuracy: 0.6110 - val_loss: 0.6358 - val_accuracy: 0.6540
Epoch 3/5
125/125 [==============================] - 33s 262ms/step - loss: 0.6199 - accuracy: 0.6600 - val_loss: 0.6718 - val_accuracy: 0.5800
Epoch 4/5
125/125 [==============================] - 33s 266ms/step - loss: 0.5824 - accuracy: 0.7045 - val_loss: 0.5943 - val_accuracy: 0.6840
Epoch 5/5
125/125 [==============================] - 34s 268ms/step - loss: 0.5171 - accuracy: 0.7530 - val_loss: 0.6247 - val_accuracy: 0.7080


In [52]:
file = [f for f in listdir(dir_test) if isfile(join(dir_test,f))]

In [ ]:
for i in range(10):
    pos = np.random.randint(0,500)
    image = cv2.imread(dir_test + file[pos])
    expand = cv2.copyMakeBorder(image,0,0,0,100,cv2.BORDER_CONSTANT)
   
    new = image.reshape(1,150,150,3)
    pre = str(model.predict_classes(new/255)[0][0])
    if pre == '0':
        text = 'cat'
    else:
        text = 'dog'
    
    cv2.putText(expand,text,(160,100),cv2.FONT_HERSHEY_SCRIPT_COMPLEX,2,(0,255,0),2)
    cv2.imshow('Pre',expand)
    cv2.waitKey()
cv2.destroyAllWindows()

In [67]:
image = cv2.imread(dir_test + file[10])


In [68]:
image = image.reshape(1,150,150,3)

In [69]:
str(model.predict_classes(image/255)[0][0])

'0'